# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# check current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra Code

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance the local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS sparkify_ks
                       WITH REPLICATION =
                       {'class':'SimpleStrategy', 'replication_factor':1}""")
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify_ks')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




---

#### *Question 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4*

To answer this question we would need to run the following query:

SELECT artist, song, length <br>
FROM session_library <br>
WHERE session_id = 338 AND item_in_session = 4

Running this query would require a table with columns for artist, song, length, session_id and item_in_session. To allow the WHERE clause to operate on session id and item in session we can set the session id as the partition key and the item in session as the clustering column. This would also create a unique primary key for each row because a particular item in session value only occurs once for each session id.

In [8]:
# Write and execute create table query
query = "CREATE TABLE IF NOT EXISTS session_library"
query = query + "(session_id int, item_in_session int, artist text, song text, length float, \
                  PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [9]:
# Extract data from consolidated csv file into a pandas dataframe
df = pd.read_csv('event_datafile_new.csv', encoding='utf8')
df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Rokia TraorÃÂ©,Stefany,F,0,White,274.88608,free,"Lubbock, TX",693,Zen,83
1,Camila,Tucker,M,1,Garrison,230.81751,free,"Oxnard-Thousand Oaks-Ventura, CA",555,Abrazame (Version Acustica),40
2,Carl Thomas,Tucker,M,0,Garrison,196.67546,free,"Oxnard-Thousand Oaks-Ventura, CA",698,You Ain't Right (Album Version),40
3,N.E.R.D.,James,M,0,Martin,242.99057,free,"Dallas-Fort Worth-Arlington, TX",78,Provider (Remix Radio Edit),79
4,Lil Jon / The East Side Boyz / DJ Flexx,Jacqueline,F,3,Lynch,285.30893,paid,"Atlanta-Sandy Springs-Roswell, GA",589,Aww Skeet Skeet,29


In [10]:
# Select columns relevant to query 1 from the event data dataframe and store the dataframe slice in a new variable
df_session_library = df[['sessionId','itemInSession','artist','song','length']]
df_session_library.head()

,sessionId,itemInSession,artist,song,length
0,693,0,Rokia TraorÃÂ©,Zen,274.88608
1,555,1,Camila,Abrazame (Version Acustica),230.81751
2,698,0,Carl Thomas,You Ain't Right (Album Version),196.67546
3,78,0,N.E.R.D.,Provider (Remix Radio Edit),242.99057
4,589,3,Lil Jon / The East Side Boyz / DJ Flexx,Aww Skeet Skeet,285.30893


In [11]:
# Extract values from the dataframe slice for insertion into the session_library table
df_session_library_records = df_session_library.values.tolist()

In [12]:
# Insert values into the session_library table
for record in df_session_library_records:
    query = "INSERT INTO session_library (session_id, item_in_session, artist, song, length)"
    query = query + "VALUES (%s, %s, %s, %s, %s)"
    try:
        session.execute(query, record)
    except Exception as e:
        print(e)

In [13]:
## Extract data from session_table to verify the data was entered into the table
query = """SELECT artist, song, length
           FROM session_library
           WHERE session_id=338 AND item_in_session=4"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


The response to the SELECT query gives us the following result:

Details of item 4 during session 338:

Artist: Faithless Music Matters <br>
Song Title: Mark Knight Dub <br>
Song Length (seconds): 495.31   

---

#### *Question 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182*

To answer this question we would need to run the following query:

SELECT artist, song, item_in_session, first_name, last_name <br>
FROM user_session_library <br>
WHERE user_id=10 and session_id=182

Running this query would require a table with columns for artist, song, item in session, the user's first name and the user's last name. To allow the WHERE clause to operate on user id and session id efficiently, we can set a combination of user id and session id as a composite partition key to help spread the data across the system. Additionally we can set the item in session as a clustering column to order the songs. This combination of keys would also create a unique primary key for each row because a particular item in session value only occurs once for each session id under a user id.

In [14]:
# Select columns relevant to query 2 from the event data dataframe and store the dataframe slice in a new variable
query = "CREATE TABLE IF NOT EXISTS user_session_library"
query = query + "(user_id int, session_id int, item_in_session int, artist text, song text, first_name text, last_name text, \
                  PRIMARY KEY ((user_id, session_id), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)                   

In [15]:
# Select columns relevant to query 2 from the event data dataframe and store the dataframe slice in a new variable
df_user_session_library = df[['userId','sessionId','itemInSession','artist','song','firstName','lastName']]
df_user_session_library.head()

,userId,sessionId,itemInSession,artist,song,firstName,lastName
0,83,693,0,Rokia TraorÃÂ©,Zen,Stefany,White
1,40,555,1,Camila,Abrazame (Version Acustica),Tucker,Garrison
2,40,698,0,Carl Thomas,You Ain't Right (Album Version),Tucker,Garrison
3,79,78,0,N.E.R.D.,Provider (Remix Radio Edit),James,Martin
4,29,589,3,Lil Jon / The East Side Boyz / DJ Flexx,Aww Skeet Skeet,Jacqueline,Lynch


In [16]:
# Extract values from the dataframe slice for insertion into the user_session_library table
df_user_session_library_records = df_user_session_library.values.tolist()

In [17]:
# Insert values into the user_session_library table
for record in df_user_session_library_records:
    query = "INSERT INTO user_session_library (user_id, session_id, item_in_session, artist, song, first_name, last_name)"
    query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
    try:
        session.execute(query, record)
    except Exception as e:
        print(e)

In [18]:
# Extract data from user_session_library table to verify the data was entered into the table 
# and to produce an output for Query 2
query = """SELECT artist, song, item_in_session, first_name, last_name 
           FROM user_session_library 
           WHERE user_id=10 and session_id=182"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, row.song, row.item_in_session, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On 0 Sylvie Cruz
Three Drives Greece 2000 1 Sylvie Cruz
Sebastien Tellier Kilometer 2 Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) 3 Sylvie Cruz


The response to the SELECT query gives us the following result:

Details of session id 338 under user id 10:

Item in Session: 0 <br>
Artist: Down To The Bone <br>
Song Title: Keep on Keepin' On <br>
User: Sylvie Cruz 

Item in Session: 1 <br>
Artist: Three Drives <br>
Song Title: Greece <br>
User: Sylvie Cruz

Item in Session: 2 <br>
Artist: Sebastien Tellier <br>
Song Title: Kilometer <br>
User: Sylvie Cruz

Item in Session: 3 <br>
Artist: Lonnie Gordon <br>
Song Title: Catch You Baby (Steve Pitron & Max Sanna Radio Edit) <br>
User: Sylvie Cruz  

---

#### *Question 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'*

To answer this question we would need to run the following query:

SELECT first_name, last_name <br>
FROM songs_by_user <br>
WHERE song='All Hands Against His Own'

Running this query would require a table with columns for first name, last name, song and user id. To allow the WHERE clause to operate on song, we can set song as the partition key. To get all the users that listened to a particular song, we should also include user id as a clustering column. This combination of keys would create a unique primary key for each user and song. Not including the user id as a clustering column would end up storing only one user per song as Cassandra would overwrite the existing user for any given song with a new user if the primary key was simple song key.  

In [19]:
# Select columns relevant to query 3 from the event data dataframe and store the dataframe slice in a new variable
query = "CREATE TABLE IF NOT EXISTS songs_by_user"
query = query + "(song text, user_id int, first_name text, last_name text, PRIMARY KEY (song, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)                 

In [20]:
# Select columns relevant to query 3 from the event data dataframe and store the dataframe slice in a new variable
df_songs_by_user = df[['song','userId','firstName','lastName']]
df_songs_by_user.head()

,song,userId,firstName,lastName
0,Zen,83,Stefany,White
1,Abrazame (Version Acustica),40,Tucker,Garrison
2,You Ain't Right (Album Version),40,Tucker,Garrison
3,Provider (Remix Radio Edit),79,James,Martin
4,Aww Skeet Skeet,29,Jacqueline,Lynch


In [21]:
# Extract values from the dataframe slice for insertion into the songs_by_user table
df_songs_by_user_records = df_songs_by_user.values.tolist()

In [22]:
# Insert values into the songs_by_user table
for record in df_songs_by_user_records:
    query = "INSERT INTO songs_by_user (song, user_id, first_name, last_name)"
    query = query + "VALUES (%s, %s, %s, %s)"
    try:
        session.execute(query, record)
    except Exception as e:
        print(e)

In [23]:
# Extract data from user_session_library table to verify the data was entered into the table 
# and to produce an output for Query 3
query = """SELECT first_name, last_name 
           FROM songs_by_user 
           WHERE song='All Hands Against His Own'"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


The response to the SELECT query gives us the following result:

Users who listened to All Hands Against His Own:

Jacqueline Lynch <br>
Tegan Levine <br>
Sara Johnson <br>

### Drop the tables before closing out the sessions

In [24]:
session.execute('DROP TABLE session_library')

In [25]:
session.execute('DROP TABLE user_session_library')

In [26]:
session.execute('DROP TABLE songs_by_user')

### Close the session and cluster connection¶

In [27]:
session.shutdown()
cluster.shutdown()